In [ ]:
from IPython.display import clear_output
from numpy import max, min

from aggr_utils import print_system_info
import tensorflow as tf
import imgaug.augmenters as iaa
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import os
import numpy as np
import matplotlib
from keras.api.layers import Input
from keras.api.models import Model, load_model
from keras.api.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
import datetime

from keras import regularizers
from keras.src.applications.mobilenet_v3 import MobileNetV3Large, preprocess_input

from keras.api.callbacks import (
    Callback,
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
    EarlyStopping,
    ReduceLROnPlateau
)

import seaborn as sns
from sklearn import metrics
from sklearn.metrics import (
    roc_curve,
    roc_auc_score,
    accuracy_score,
    classification_report,
    confusion_matrix
)
import math

In [ ]:
print_system_info()
seed = 73
IMG_SIZE = 224
COLOR_CHANNELS = 3
FRAMES_PER_SECOND = 2
DATASET_DIR = '../datasets/violence'
KERAS_MODEL_FILENAME = 'mobilenet_v3_model.keras'
CHECKPOINT_FILEPATH = "ModelWeights_v3.weights.h5"
tf.random.set_seed(seed)

In [ ]:
TPU_INIT = False

if TPU_INIT:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime!')
else:
    !nvidia-smi


## **Preprocessing**
+ **Getting frames form video**
+ **some image argumentations**

In [ ]:
def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)

    frame_rate = vidcap.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(frame_rate // FRAMES_PER_SECOND)
    count = 0

    ImageFrames = []
    while vidcap.isOpened():
        frame_id = vidcap.get(1)
        success, image = vidcap.read()

        if success:
            if frame_id % frames_to_skip == 0:
                flip = iaa.Fliplr(1.0)
                zoom = iaa.Affine(scale=1.3)
                random_brightness = iaa.Multiply((1, 1.3))
                rotate = iaa.Affine(rotate=(-25, 25))

                image_aug = flip(image=image)
                image_aug = random_brightness(image=image_aug)
                image_aug = zoom(image=image_aug)
                image_aug = rotate(image=image_aug)

                rgb_img = cv2.cvtColor(image_aug, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)

            count += 1
        else:
            break

    vidcap.release()

    return ImageFrames

In [ ]:
video_data_dir = os.path.join(DATASET_DIR, 'Real Life Violence Dataset')

print('Доступно \n{} видео с агрессией \n{} видео без агрессии'.format(
    len(os.listdir(video_data_dir + '/Violence')),
    len(os.listdir(video_data_dir + '/NonViolence'))))

x_original = []
y_original = []

# Выбор случайных 1000 видео для решения проблем с памятью
# 500 с агрессией и 500 без
VIDEOS_PER_CLASS = 500
CLASSES = ["NonViolence", "Violence"]

for category in CLASSES:
    path = os.path.join(video_data_dir, category)
    class_num = CLASSES.index(category)
    for i, video in enumerate(tqdm(os.listdir(path)[VIDEOS_PER_CLASS:])):
        frames = video_to_frames(os.path.join(path, video))
        for j, frame in enumerate(frames):
            x_original.append(frame)
            y_original.append(class_num)

In [ ]:
x_original = np.array(x_original).reshape(-1, IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_original)
print(len(x_original))

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=seed)

for train_index, test_index in stratified_sample.split(x_original, y_original):
    x_train, x_test = x_original[train_index], x_original[test_index]
    y_train, y_test = y_original[train_index], y_original[test_index]


def preprocess_input(x):
    x = x.reshape(-1, IMG_SIZE, IMG_SIZE, COLOR_CHANNELS) / 255.0
    # Модель ожидает на вход от -1.0 до 1.0
    return (x - 0.5) * 2.0


x_train_nn = preprocess_input(x_train)
x_test_nn = preprocess_input(x_test)

## **Model Training**

In [ ]:
clear_output()

In [ ]:
matplotlib.use("Agg")
print(min(x_train_nn))
print(max(x_train_nn))

In [ ]:
epochs = 50
kernel_regularizer = regularizers.l2(0.0001)
FROM_FILE = False


def load_layers():
    input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, COLOR_CHANNELS))
    baseModel = MobileNetV3Large(
        pooling='avg',
        include_top=False,
        include_preprocessing=False,
        input_tensor=input_tensor,
        input_shape=(IMG_SIZE, IMG_SIZE, COLOR_CHANNELS)
    )

    print(input_tensor)
    headModel = baseModel.output
    headModel = Dense(1, activation="sigmoid", kernel_regularizer=kernel_regularizer)(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False

    print("Compiling model...")
    model.compile(loss="binary_crossentropy",
                  optimizer='adam',
                  metrics=["accuracy"])

    return model


if FROM_FILE:
    model = load_model(CHECKPOINT_FILEPATH)
else:
    if TPU_INIT:
        with tpu_strategy.scope():
            model = load_layers()
    else:
        model = load_layers()

model.summary()

In [ ]:
PATIENCE = 3

START_LR = 0.00001
MIN_LR = 0.00001
MAX_LR = 0.00005

BATCH_SIZE = 4
RAMP_UP_EPOCHS = 5
SUSTAIN_EPOCHS = 0
EXP_DECAY = .8
EARLY_STOP_ACCURACY = 0.999

if TPU_INIT:
    MAX_LR *= tpu_strategy.num_replicas_in_sync
    BATCH_SIZE *= tpu_strategy.num_replicas_in_sync


def lr_fn(epoch):
    if epoch < RAMP_UP_EPOCHS:
        lr = (MAX_LR - START_LR) / RAMP_UP_EPOCHS * epoch + START_LR
    elif epoch < RAMP_UP_EPOCHS + SUSTAIN_EPOCHS:
        lr = MAX_LR
    else:
        decay_epoch = epoch - RAMP_UP_EPOCHS - SUSTAIN_EPOCHS
        lr = (MAX_LR - MIN_LR) * (EXP_DECAY ** decay_epoch) + MIN_LR
    return lr


class EarlyStopCallback(Callback):
    def __init__(self, target_acc=EARLY_STOP_ACCURACY):
        super().__init__()
        self.target_acc = target_acc

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy')

        if accuracy >= self.target_acc:
            print("\nДостигнут лимит остановки обучения!")
            self.model.stop_training = True

In [ ]:
def build_callbacks(checkpoint_filepath=CHECKPOINT_FILEPATH, log_dir="logs/fit/"):
    log_dir = os.path.join(log_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks = []

    callbacks.append(
        EarlyStopCallback()
    )

    callbacks.append(
        LearningRateScheduler(
            lambda epoch: lr_fn(epoch),
            verbose=False
        )
    )

    callbacks.append(
        ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1,
            save_best_only=True
        )
    )

    if not TPU_INIT:
        callbacks.append(
            TensorBoard(
                log_dir=log_dir,
                write_graph=True,
                histogram_freq=1
            )
        )

    callbacks.append(
        EarlyStopping(
            patience=PATIENCE,
            monitor='val_loss',
            mode='min',
            restore_best_weights=True,
            verbose=1,
            min_delta=.00075
        )
    )

    callbacks.append(
        ReduceLROnPlateau(
            patience=2,
            mode='min'
        )
    )

    return callbacks


os.system('rm -rf ./logs/')
callbacks = build_callbacks()

In [ ]:
def train_model(_model):
    print("Начало обучения головы модели")
    history = _model.fit(
        x_train_nn,
        y_train,
        validation_data=(x_test_nn, y_test),
        epochs=epochs,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    print(f'\nВосстановление лучших весов для MobileNetV3 из {CHECKPOINT_FILEPATH}')
    _model.load_weights(CHECKPOINT_FILEPATH)

    return history

In [ ]:
history = train_model(model)

In [ ]:
%matplotlib inline

def print_graph(history, metric, best_epoch, save=True):
    plt.figure()

    plt.plot(history.history[metric], label='Train')
    plt.plot(history.history[f'val_{metric}'], label='Val')

    if best_epoch is not None:
        plt.axvline(best_epoch, linestyle='--', color='gray', alpha=0.6, label='Best Epoch')

    plt.title(f'{metric.capitalize()} по эпохам')
    plt.xlabel('Epoch')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)

    if save:
        plot = '{}.png'.format(metric)
        plt.savefig(plot)
    plt.show()


def model_summary(_model, _history, _x_test, _y_test):
    print('---' * 30)
    test_loss, test_accuracy = _model.evaluate(_x_test, _y_test, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.4f} | Test Loss: {test_loss:.4f}")

    # Определим лучшую эпоху по минимальному val_loss
    if _history:
        val_losses = _history.history['val_loss']
        best_epoch = int(np.argmin(val_losses))

        print(f'Лучшая эпоха: {best_epoch}')
        print(f"Train Acc: {_history.history['accuracy'][best_epoch]:.4f} | "
              f"Train Loss: {_history.history['loss'][best_epoch]:.4f}")

        print_graph(_history, 'accuracy', best_epoch)
        print_graph(_history, 'loss', best_epoch)
        print('---' * 30)

In [ ]:
model_summary(model, history, x_test_nn, y_test)

## **Evaluation on test set**

In [ ]:
# evaluate the network
print("Проверка модели...")
predictions = model.predict(x_test_nn)
preds = predictions > 0.5
corr_pred = metrics.confusion_matrix(y_test, preds)

n_correct = np.uint64((corr_pred[0][0] + corr_pred[1][1]))
print('> Correct Predictions:', n_correct)
n_wrongs = np.uint((corr_pred[0][1] + (corr_pred[1][0])))
print('> Wrong Predictions:', n_wrongs)

sns.heatmap(corr_pred, annot=True, fmt="d", cmap="Blues")
plt.show()

print(metrics.classification_report(y_test, preds,
                                    target_names=["NonViolence", "Violence"]))

In [ ]:
def evaluate_model(_model, _x_test, _y_test, threshold=0.5):
    print("\n=== Проверка модели ===")
    _predictions = _model.predict(_x_test, batch_size=64, verbose=1).flatten()
    predictions_binary = int(_predictions > threshold)

    # Confusion matrix
    cm = metrics.confusion_matrix(_y_test, predictions_binary)
    correct = np.sum(np.diag(cm))
    total = np.sum(cm)

    print(f"> Correct Predictions: {correct}")
    print(f"> Wrong Predictions: {total - correct}")
    print("\nConfusion Matrix:")
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=["NonViolence", "Violence"],
                yticklabels=["NonViolence", "Violence"])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    print("\nClassification Report:")
    print(metrics.classification_report(_y_test, predictions_binary,
                                        target_names=["NonViolence", "Violence"]))

    auc = metrics.roc_auc_score(_y_test, _predictions)
    print(f"ROC-AUC: {auc:.4f}")

    # ROC Curve
    fpr, tpr, _ = metrics.roc_curve(_y_test, _predictions)
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f'AUC = {auc:.4f}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.grid(True)
    plt.show()


evaluate_model(model, x_test_nn, y_test)

In [ ]:
model.save(KERAS_MODEL_FILENAME)